In [6]:
import httpx
import asyncio
import nest_asyncio
from datetime import datetime, timedelta
from telegram import Bot

# Enable asyncio in a Jupyter environment
nest_asyncio.apply()

# Configuration
BOT_TOKEN = '6593706311:AAGfep182h5A1hOifip4095pVEy4Rp7kZTs'  # Your Telegram Bot Token
CHAT_ID = "@roofpocazz"  # Your chat ID
BITFINEX_API_URL = 'https://api.bitfinex.com/v2/ticker/'
symbols = ['BTCUST', 'ETHUST', 'XRPUST', 'LTCUST', 'ADAUST', 'SOLUST', 'XMRUST']
##'LINKUST',

In [7]:
async def get_current_price(symbol):
    async with httpx.AsyncClient() as client:
        url = f'{BITFINEX_API_URL}t{symbol}'
        response = await client.get(url)
        
        if response.status_code == 200:
            try:
                data = response.json()
                price = data[6]  # Last traded price is the 7th item in the returned list
                return price
            except Exception as e:
                print('Error processing response:', e)
                print(response.content)  # Print the full response for troubleshooting
        else:
            print(f'Failed to fetch data for {symbol}. Status code: {response.status_code}')
            return None

In [8]:
async def send_telegram_message(message):
    bot = Bot(token=BOT_TOKEN)
    await bot.send_message(chat_id=CHAT_ID, text=message)

In [9]:
async def fetch_and_send_prices():
    gmt_plus_6_time = datetime.utcnow() + timedelta(hours=6)
    message = f'BITFINEX: Prices at {gmt_plus_6_time.strftime("%Y-%m-%d %H:%M:%S")} (GMT+6):\n\n'
    
    for symbol in symbols:
        price = await get_current_price(symbol)
        if price is not None:
            formatted_price = f'{float(price):.2f}'
            
            # Replace 'UST' with 'USDT' for output
            display_symbol = symbol.replace("UST", "USDT")
            message += f'{display_symbol}: ${formatted_price}\n'
    
    await send_telegram_message(message)

In [10]:
# Run the asyncio event loop
async def main():
    await fetch_and_send_prices()

loop = asyncio.get_event_loop()
loop.run_until_complete(main())